In [1]:
import cv2
import dlib
import time


In [2]:
# Descargar herramientas de c++https://visualstudio.microsoft.com/es/visual-cpp-build-tools/
# Descargar el modelo desde: http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2

# Ruta al modelo preentrenado de predicción de 68 puntos de landmarks
predictor_path = "shape_predictor_68_face_landmarks.dat"

# Cargar el detector de rostros y el predictor de landmarks
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

In [ ]:
import cv2
import dlib
import numpy as np

# Ruta al modelo de predicción de landmarks faciales
predictor_path = "shape_predictor_68_face_landmarks.dat"

# Cargar el detector de rostros y el predictor de landmarks
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)
video_capture = cv2.VideoCapture(0)

# Cargar la imagen del sombrero de Chopper con fondo transparente
chopper_hat = cv2.imread("/mnt/data/chopper_hat.png", cv2.IMREAD_UNCHANGED)  # Ruta completa al sombrero de Chopper

def overlay_image(background, overlay, pos_x, pos_y):
    """
    Superpone una imagen con canal alfa sobre otra imagen en una posición específica.
    """
    h, w = overlay.shape[:2]
    if pos_y + h >= background.shape[0] or pos_x + w >= background.shape[1]:
        return background  # No dibujar si se sale de la imagen
    
    # Recortar si el overlay se sale de los límites del frame
    overlay_crop = overlay
    if pos_y + h > background.shape[0]:
        overlay_crop = overlay_crop[:background.shape[0] - pos_y, :]
    if pos_x + w > background.shape[1]:
        overlay_crop = overlay_crop[:, :background.shape[1] - pos_x]
    
    alpha_overlay = overlay_crop[:, :, 3] / 255.0  # Canal alfa para transparencia
    for c in range(0, 3):
        background[pos_y:pos_y+overlay_crop.shape[0], pos_x:pos_x+overlay_crop.shape[1], c] = \
            (1 - alpha_overlay) * background[pos_y:pos_y+overlay_crop.shape[0], pos_x:pos_x+overlay_crop.shape[1], c] + \
            alpha_overlay * overlay_crop[:, :, c]
    return background

while True:
    ret, frame = video_capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    for face in faces:
        landmarks = predictor(gray, face)

        # Obtener puntos clave de los ojos y el mentón para el tamaño y posición del sombrero
        left_eye = (landmarks.part(36).x, landmarks.part(36).y)
        right_eye = (landmarks.part(45).x, landmarks.part(45).y)
        chin = (landmarks.part(8).x, landmarks.part(8).y)

        # Calcular escala para que el sombrero cubra toda la cabeza
        head_width = int(1.8 * (right_eye[0] - left_eye[0]))  # Anchura del sombrero
        head_height = int(1.3 * abs(chin[1] - left_eye[1]))   # Altura del sombrero
        
        # Redimensionar el sombrero de Chopper
        resized_chopper_hat = cv2.resize(chopper_hat, (head_width, head_height))
        
        # Posición del sombrero: centrado sobre la frente
        hat_pos_x = left_eye[0] - head_width // 4
        hat_pos_y = left_eye[1] - head_height

        # Superponer el sombrero en la imagen
        overlay_image(frame, resized_chopper_hat, hat_pos_x, hat_pos_y)

    # Mostrar la imagen con el sombrero de Chopper superpuesto
    cv2.imshow("Filtro de One Piece", frame)
    if cv2.waitKey(5) & 0xFF == ord("q"):
        break

video_capture.release()
cv2.destroyAllWindows()


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


: 